In [1]:
import pandas as pd
import numpy as np
import json
import os

In [2]:
df=pd.read_json('amsterdam.json')

In [3]:
df.head()

,host_listings_count,accommodates,bathrooms,bedrooms,guests_included,minimum_nights,number_of_reviews,calculated_host_listings_count,price,latitude,longitude,room_type,instant_bookable
0,1.0,6,1.0,2.0,1,3,4,1,$145.00,52.361173,4.866755,Entire home/apt,f
1,1.0,2,1.0,1.0,1,2,17,1,$88.00,52.370237,4.859507,Entire home/apt,f
2,1.0,2,1.0,1.0,2,2,4,1,$130.00,52.362016,4.866872,Entire home/apt,f
3,2.0,4,1.0,2.0,2,4,2,2,$110.00,52.365826,4.858178,Entire home/apt,f
4,1.0,2,1.0,1.0,1,3,10,1,$115.00,52.364179,4.864885,Entire home/apt,f


In [4]:
print(df.shape)
print(df.columns)

(15181, 13)
Index(['host_listings_count', 'accommodates', 'bathrooms', 'bedrooms',
       'guests_included', 'minimum_nights', 'number_of_reviews',
       'calculated_host_listings_count', 'price', 'latitude', 'longitude',
       'room_type', 'instant_bookable'],
      dtype='object')


In [5]:
df.isnull().sum()

host_listings_count                3
accommodates                       0
bathrooms                         18
bedrooms                          12
guests_included                    0
minimum_nights                     0
number_of_reviews                  0
calculated_host_listings_count     0
price                              0
latitude                           0
longitude                          0
room_type                          0
instant_bookable                   0
dtype: int64

In [6]:
df.describe()

,host_listings_count,accommodates,bathrooms,bedrooms,guests_included,minimum_nights,number_of_reviews,calculated_host_listings_count,latitude,longitude
count,15178.000000,15181.000000,15163.000000,15169.000000,15181.000000,15181.000000,15181.000000,15181.000000,15181.000000,15181.000000
mean,3.404994,2.917858,1.122832,1.412222,1.516106,2.882551,17.785456,2.934787,52.365353,4.889025
std,11.324173,1.404973,0.356547,0.885127,1.004709,7.215722,34.747999,8.968696,0.015738,0.034901
min,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,52.290308,4.753513
25%,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,52.355616,4.865016
50%,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,7.000000,1.000000,52.365028,4.886344
75%,1.000000,4.000000,1.000000,2.000000,2.000000,3.000000,18.000000,1.000000,52.374829,4.907857
max,106.000000,17.000000,8.000000,10.000000,16.000000,523.000000,496.000000,91.000000,52.426224,5.027689


In [7]:
max(df['price'].values,key=len)

'$1,305.00'

In [8]:
df['price']=df['price'].apply(lambda x:x.replace('$',''))
df['price']=df['price'].apply(lambda x:x.replace(',',''))

df['price']=pd.to_numeric(df['price'])

In [9]:
print(df['price'].head())

0    145.0
1     88.0
2    130.0
3    110.0
4    115.0
Name: price, dtype: float64


import gmaps
fig = gmaps.Map(layout={'width': '1000px', 'height': '500px', 'padding': '10px'})
fig.add_layer(gmaps.heatmap_layer(df[['latitude', 'longitude']],
 weights=df['price']))
fig

In [10]:
df.dtypes

host_listings_count               float64
accommodates                        int64
bathrooms                         float64
bedrooms                          float64
guests_included                     int64
minimum_nights                      int64
number_of_reviews                   int64
calculated_host_listings_count      int64
price                             float64
latitude                          float64
longitude                         float64
room_type                          object
instant_bookable                   object
dtype: object

In [11]:
df=df[(df.price<=440.0)]
df=df.dropna()

In [12]:
a_latitude=52.379189
a_longitude=4.899431

In [13]:
# Find the distance of all observations in the data from the city centre in km unitw
#Assuming city centre has latitute = 52.379189 and longitude = 4.899431
from math import radians, cos, sin, asin, sqrt
def haversine(lon1,lon2,lat1,lat2):
    """
    Calculate the great circle distance between 2 points
    on the earth(specified in decimal degrees)
    """
    #convert decimal degrees to radians
    lon1, lon2, lat1, lat2 = map(radians, [lon1,lon2,lat1,lat2])
    #haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2+cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c= 2* asin(sqrt(a))
    km=6367*c
    return km

for index, row in df.iterrows():
    df.loc[index,'distance'] = haversine(a_latitude, a_longitude, row['longitude'], row['latitude'])

In [14]:
df.head()

,host_listings_count,accommodates,bathrooms,bedrooms,guests_included,minimum_nights,number_of_reviews,calculated_host_listings_count,price,latitude,longitude,room_type,instant_bookable,distance
0,1.0,6,1.0,2.0,1,3,4,1,145.0,52.361173,4.866755,Entire home/apt,f,6825.027540
1,1.0,2,1.0,1.0,1,2,17,1,88.0,52.370237,4.859507,Entire home/apt,f,6826.271424
2,1.0,2,1.0,1.0,2,2,4,1,130.0,52.362016,4.866872,Entire home/apt,f,6825.067635
3,2.0,4,1.0,2.0,2,4,2,2,110.0,52.365826,4.858178,Entire home/apt,f,6826.129438
4,1.0,2,1.0,1.0,1,3,10,1,115.0,52.364179,4.864885,Entire home/apt,f,6825.389113


In [15]:
df.dtypes

host_listings_count               float64
accommodates                        int64
bathrooms                         float64
bedrooms                          float64
guests_included                     int64
minimum_nights                      int64
number_of_reviews                   int64
calculated_host_listings_count      int64
price                             float64
latitude                          float64
longitude                         float64
room_type                          object
instant_bookable                   object
distance                          float64
dtype: object

#Filter roomtype = Private room and distance <1
df= df['room_type','instant_bookable'].astype(str).astype(int)
#df.to_numeric(df["room_type","instant_bookable"])

dt3=df[df.room_type=='Private room']
dt3=dt3[dt3.distance<=1]
dt3.price.describe()

# Modelling

we will prepare our dataset by encoding the categorical variable room_type and by splitting it into train and test

In [16]:
df=pd.get_dummies(df)

X=df.drop(['price'],axis=1)
y=df['price']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=1)

# Baseline model 📏:
The first model used will be used as a baseline, as we need a benchmark to assess the performances of other models and compare results.
It consists of a classical Linear Regression, evaluated with cross-validation using the GridSearchCV class on r2 and MAE metrics.

In [17]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()

lin_reg.fit(X_train,y_train)
y_pred =lin_reg.predict(X_test)

We can now calculate our r2 and MAE errors by creating a dummy dataframe to store results for each model:

In [18]:
from sklearn import metrics

r2= metrics.r2_score(y_test,y_pred)
mae = metrics.mean_absolute_error(y_test,y_pred)

scores = pd.DataFrame({'Baseline (regression)':[r2,mae]},index=['R2','MAE'])

scores

,Baseline (regression)
R2,0.401728
MAE,35.439306


The mae tells us that our predictions are on average 35$ off, while the R2 tells us our data are pretty sparsed.

The next model has been chosen according to the sklearn map, and it consists of a Support Vector Machine. However, since playing with parameters its not always easy, and it might require specific knowledge GridSearchCV will come in help:

# Support Vector Machine Model 📈:

In [ ]:
if 'svr_gridsearch_cv.pkl' in os.listdir():
    svr_grid_search = joblib.load('svr_gridsearch_cv.pkl')
    
else:
    from sklearn.svm import SVR
    
svr = SVR()
from sklearn.model_selection import GridSearchCV

param_grid = [
      {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
      {'C': [1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001], 'kernel': ['rbf']}]

svr_grid_search = GridSearchCV(svr, param_grid=param_grid, 
                                   n_jobs=-1, 
                                   scoring=['r2', 'neg_mean_squared_error'],
                                  refit='neg_mean_squared_error', verbose=100)

svr_grid_search.fit(X_train,y_train)
joblib.dump(svr_grid_search.best_estimator_,'svr_gridsearch_cv.pkl')

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(16,), dtype=object).
Pickling array (shape=(11998,), dtype=int64).
Pickling array (shape=(6, 11998), dtype=float64).
Pickling array (shape=(5, 11998), dtype=int64).
Pickling array (shape=(5, 11998), dtype=uint8).
Pickling array (shape=(6,), dtype=object).
Pickling array (shape=(5,), dtype=object).
Pickling array (shape=(5,), dtype=object).
Pickling array (shape=(6,), dtype=int64).
Pickling array (shape=(5,), dtype=int64).
Pickling array (shape=(11998,), dtype=int64).
Pickling array (shape=(11998,), dtype=float64).
Pickling array (shape=(11998,), dtype=int64).
Pickling array (shape=(9598,), dtype=int32).
Pickling array (shape=(2400,), dtype=int32).
Pickling array (shape=(16,), dtype=object).
Pickling array (shape=(11998,), dtype=int64).
Pickling array (shape=(6, 11998), dtype=float64).
Pickling array (shape=(5, 11998), dtype=int6

In [ ]:
!conda install -c conda-forge lightgbm

In [ ]:
if 'gbm_gridsearch_cv.pkl' in os.listdir():
    gbm_grid_search = joblib.load('gbm_gridsearch_cv.pkl')
    
else:
    from lightgbm import LGBMRegressor
    
gbm= LGBMRegressor()

param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [50, 100, 150],
    'boosting_type': ['gbdt', 'dart'],
    'num_leaves': [15, 31, 50]}

gbm_grid_search = GridSearchCV(gbm, param_grid=param_grid, 
                                   n_jobs=-1, 
                                   scoring=['r2', 'neg_mean_squared_error'],
                                  refit='neg_mean_squared_error', verbose=100)

gbm_grid_search.fit(X_train,y_train)

joblib.dump(gbm_grid_search.best_estimator_,'gbm_gridsearch_cv.pkl')

# Neural Network 🧠:

In [ ]:
def build_model():
    model = keras.Sequential([
     tf.keras.layers.Dense(64, activation=’relu’, input_shape=(25,)), 
     tf.keras.layers.Dropout(0.2), 
     tf.keras.layers.Dense(128, activation=’relu’),
     tf.keras.layers.Dropout(0.2), 
     tf.keras.layers.Dense(1)
     ])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model.compile(loss=’mean_squared_error’,
     optimizer=optimizer,
     metrics=[‘mae’, r2_keras])
     return model

In [ ]:
coefficients = pd.concat([pd.DataFrame(X.columns, columns=['variable']), 
pd.DataFrame(np.transpose(lin_reg.coef_), columns ['coefficients'])], axis = 1)
coefficients